In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List
import warnings
warnings.filterwarnings('ignore')

# Set up directories
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")
TRANSFORMED_DIR = Path("../data/transformed")

# Create directories if they don't exist
for dir_path in [PROCESSED_DIR, TRANSFORMED_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

def transform_ts_data_into_features_and_target(
    df: pd.DataFrame,
    feature_col: str = "ride_count",
    weather_cols: List[str] = None,
    rides_window_size: int = 672,  # 28 days * 24 hours = 672
    weather_window_size: int = 3,   # 3 hours of weather data
    step_size: int = 1,
    top_n_stations: int = 50
) -> pd.DataFrame:
    """
    Transform time series data into features and target using sliding window approach.
    Based on the reference code pattern.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with time series data
        feature_col (str): Main feature column for rides
        weather_cols (List[str]): Weather columns to include
        rides_window_size (int): Window size for rides (28 days = 672 hours)
        weather_window_size (int): Window size for weather data
        step_size (int): Step size for sliding window
    
    Returns:
        pd.DataFrame: Transformed DataFrame with features and target
    """
    
    if weather_cols is None:
        weather_cols = ["temp", "wind_speed", "precipitation"]
    
    print("Starting sliding window transformation...")
    print(f"Rides window size: {rides_window_size} (28 days)")
    print(f"Weather window size: {weather_window_size} hours")
    print(f"Step size: {step_size}")
    
    station_totals = (
        df.groupby("start_station_id")[feature_col]
        .sum()
        .sort_values(ascending=False)
    )
    top_stations = station_totals.head(top_n_stations).index
    df = df[df["start_station_id"].isin(top_stations)].copy()

    print(f"Number of stations after filtering: {len(top_stations)}")

    # Get all unique station IDs
    station_ids = df["start_station_id"].unique()
    print(f"Processing {len(station_ids)} stations...")
    
    # List to store transformed data for each station
    all_transformed_data = []
    
    for station_id in station_ids:
        try:
            # Filter data for current station and sort by hour
            station_data = df[df["start_station_id"] == station_id].copy()
            station_data = station_data.sort_values("hour").reset_index(drop=True)
            
            # Skip if not enough data
            if len(station_data) <= rides_window_size:
                print(f"Skipping station {station_id}: insufficient data ({len(station_data)} records)")
                continue
            
            # Extract values as arrays
            ride_values = station_data[feature_col].values
            hour_values = station_data["hour"].values
            
            # Extract weather values
            weather_values = {}
            for col in weather_cols:
                if col in station_data.columns:
                    weather_values[col] = station_data[col].values
            
            # Create sliding windows
            rows = []
            for i in range(0, len(ride_values) - rides_window_size, step_size):
                # Rides features (last 28 days = 672 hours)
                ride_features = ride_values[i:i + rides_window_size]
                
                # Weather features (last 3 hours for each weather variable)
                weather_features = []
                for col in weather_cols:
                    if col in weather_values:
                        # Get the last 3 hours of weather data (aligned with the end of rides window)
                        weather_data = weather_values[col][i + rides_window_size - weather_window_size:i + rides_window_size]
                        weather_features.extend(weather_data)
                
                # Target (next hour's ride count)
                target = ride_values[i + rides_window_size]
                
                # Timestamp for the target
                target_hour = hour_values[i + rides_window_size]
                
                # Station ID
                station_id_val = station_id
                
                # Combine all features and metadata
                all_features = np.concatenate([ride_features, np.array(weather_features)])
                row = np.append(all_features, [target, station_id_val, target_hour])
                rows.append(row)
            
            # Create column names
            ride_feature_cols = [f"rides_t-{rides_window_size - j}" for j in range(rides_window_size)]
            
            weather_feature_cols = []
            for col in weather_cols:
                if col in weather_values:
                    weather_feature_cols.extend([f"{col}_t-{weather_window_size - k}" for k in range(weather_window_size)])
            
            all_feature_cols = ride_feature_cols + weather_feature_cols
            all_columns = all_feature_cols + ["target","start_station_id", "hour"]
            
            # Create DataFrame for this station
            if rows:  # Check if we have any rows
                station_transformed = pd.DataFrame(rows, columns=all_columns)
                all_transformed_data.append(station_transformed)
                
                if len(all_transformed_data) % 10 == 0:  # Print progress every 10 stations
                    print(f"Processed {len(all_transformed_data)} stations...")
            
        except Exception as e:
            print(f"Error processing station {station_id}: {e}")
            continue
    
    if not all_transformed_data:
        raise ValueError("No data could be transformed. Check window sizes and data availability.")
    
    # Combine all stations
    final_df = pd.concat(all_transformed_data, ignore_index=True)
    
    print(f"Transformation complete!")
    print(f"Final dataset: {len(final_df)} records")
    print(f"Number of features: {len(all_feature_cols)}")
    print(f"  - Ride features: {len(ride_feature_cols)}")
    print(f"  - Weather features: {len(weather_feature_cols)}")
    
    return final_df

def store_transformed_data(transformed_df: pd.DataFrame, filename: str = "transformed_features_and_target_top50.parquet"):
    """Store the transformed features and target data"""
    
    output_path = TRANSFORMED_DIR / filename
    transformed_df.to_parquet(output_path, index=False)
    
    print(f"Stored transformed data to: {output_path}")
    print(f"Dataset shape: {transformed_df.shape}")
    
    return transformed_df

def analyze_transformed_data(transformed_df: pd.DataFrame):
    """Analyze the transformed dataset"""
    
    print("\n" + "="*60)
    print("TRANSFORMED DATA ANALYSIS")
    print("="*60)
    
    print(f"Total records: {len(transformed_df):,}")
    print(f"Number of stations: {transformed_df['start_station_id'].nunique()}")
    print(f"Time range: {transformed_df['hour'].min()} to {transformed_df['hour'].max()}")
    
    # Count feature types
    ride_features = [col for col in transformed_df.columns if 'rides_t-' in col]
    weather_features = [col for col in transformed_df.columns if any(w in col for w in 
                        ['temp', 'wind_speed', 'precipitation'])]
    
    print(f"\nFeature breakdown:")
    print(f"  Ride features: {len(ride_features)}")
    print(f"  Weather features: {len(weather_features)}")
    print(f"  Total features: {len(transformed_df.columns) - 3}")  # Excluding target, station_id, hour
    
    # Check for missing values
    missing_values = transformed_df.isnull().sum().sum()
    print(f"Missing values: {missing_values}")
    
    # Target statistics
    print(f"\nTarget statistics:")
    print(f"  Min: {transformed_df['target'].min()}")
    print(f"  Max: {transformed_df['target'].max()}")
    print(f"  Mean: {transformed_df['target'].mean():.2f}")
    print(f"  Std: {transformed_df['target'].std():.2f}")
    
    # Show sample
    print(f"\nSample of transformed data:")
    sample_cols = ['target', 'start_station_id', 'hour'] + transformed_df.columns[:5].tolist()
    print(transformed_df[sample_cols].head())

# MAIN EXECUTION
if __name__ == "__main__":
    print("Starting Blue Bikes Data Transformation Pipeline")
    print("=" * 50)
    
    try:
        # Load your merged data (adjust the path as needed)
        processed_file = PROCESSED_DIR / "rides_weather_combined.parquet"
        
        if processed_file.exists():
            print("Loading processed data...")
            merged_data = pd.read_parquet(processed_file)
            print(f"Loaded data: {len(merged_data):,} records")
            print(f"Columns: {list(merged_data.columns)}")
        else:
            # If no processed file exists, use a sample (you'll need to replace this with your actual data loading)
            print("Processed file not found. Please ensure you have run the data merging step first.")
            print("Creating sample data for demonstration...")
            
            # Sample data structure - REPLACE THIS WITH YOUR ACTUAL DATA LOADING
            dates = pd.date_range("2024-01-01", "2024-12-31", freq="H")
            sample_data = []
            for station_id in ["A32000", "A32001", "A32002"]:
                for date in dates:
                    sample_data.append({
                        'hour': date,
                        'start_station_id': station_id,
                        'ride_count': np.random.randint(0, 10),
                        'temp': np.random.uniform(30, 90),
                        'humidity': np.random.uniform(30, 90),
                        'pressure': np.random.uniform(1000, 1020),
                        'wind_speed': np.random.uniform(0, 20),
                        'wind_deg': np.random.uniform(0, 360),
                        'precipitation': np.random.uniform(0, 5)
                    })
            merged_data = pd.DataFrame(sample_data)
            print(f"Created sample data: {len(merged_data):,} records")
        
        # Transform the data
        print("\n" + "="*50)
        print("TRANSFORMING DATA WITH SLIDING WINDOW")
        print("="*50)
        
        transformed_data = transform_ts_data_into_features_and_target(
            df=merged_data,
            feature_col="ride_count",
            rides_window_size=672,  # 28 days
            weather_window_size=3,   # 3 hours
            step_size=1
        )
        
        # Store the transformed data
        print("\n" + "="*50)
        print("STORING TRANSFORMED DATA")
        print("="*50)
        
        final_data = store_transformed_data(transformed_data)
        
        # Analyze the results
        print("\n" + "="*50)
        print("ANALYSIS")
        print("="*50)
        
        analyze_transformed_data(final_data)
        
        print("\n🎉 TRANSFORMATION COMPLETED SUCCESSFULLY!")
        print("The dataset is now ready for machine learning model training.")
        
    except Exception as e:
        print(f"❌ Error in transformation pipeline: {e}")
        raise

print("\nCode execution completed!")

Starting Blue Bikes Data Transformation Pipeline
Loading processed data...
Loaded data: 5,187,319 records
Columns: ['hour', 'start_station_id', 'ride_count', 'is_weekend', 'is_us_holiday', 'lat_round', 'lon_round', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg', 'precipitation']

TRANSFORMING DATA WITH SLIDING WINDOW
Starting sliding window transformation...
Rides window size: 672 (28 days)
Weather window size: 3 hours
Step size: 1
Number of stations after filtering: 50
Processing 50 stations...
Processed 10 stations...
Processed 20 stations...
Processed 30 stations...
Processed 40 stations...
Processed 50 stations...
Transformation complete!
Final dataset: 404584 records
Number of features: 681
  - Ride features: 672
  - Weather features: 9

STORING TRANSFORMED DATA
Stored transformed data to: ../data/transformed/transformed_features_and_target_top50.parquet
Dataset shape: (404584, 684)

ANALYSIS

TRANSFORMED DATA ANALYSIS
Total records: 404,584
Number of stations: 50
Time r